In [2]:
from keras.datasets import imdb
import numpy as np
from sklearn.decomposition import NMF
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Activation, Bidirectional, Add, Concatenate, Embedding, Lambda, concatenate, Dropout, TimeDistributed
from keras.layers import LSTM, Multiply, Lambda,TimeDistributed,GRU
from keras.optimizers import Adam
from keras.layers.core import Masking
from keras import backend as K
from keras import losses
import pandas as pd
import tensorflow as tf
from dateutil.parser import parse
import datetime
import copy
from keras.layers import LeakyReLU

In [122]:
embed_size = 10
hidden_size = 20
sequential_code_channel_size=3
logical_variables = [1,2]
logical_operator = {
    3:'or',
    4:'and'
}
none_var = 0
num_vocab=5
batch_size=50
episode_len = 10
num_epoch = 1000000

In [123]:
def get_encoder_agent():
    x_i = Input(shape=(None,1))
    x_o = Input(shape=(None,1))

    y = TimeDistributed(Embedding(num_vocab, embed_size))(x_i)
    y = Lambda(lambda z:K.squeeze(z,axis=-2))(y)
    print(K.int_shape(y))
    y,h,c = LSTM(hidden_size,return_state=True)(y)
    
    y,_,_ = LSTM(hidden_size,return_sequences=True, return_state=True)(x_o,initial_state=[h,c])
    
    y = TimeDistributed(Dense(sequential_code_channel_size,activation='softmax'))(y)    
    
    model = Model(inputs=[x_i,x_o],outputs=y)
    
    return model

In [88]:
def get_decoder_agent():
    x_i = Input(shape=(None,1))
    x_o = Input(shape=(None,1))

    y = TimeDistributed(Embedding(sequential_code_channel_size, embed_size))(x_i)
    y = Lambda(lambda z:K.squeeze(z,axis=-2))(y)
    print(K.int_shape(y))
    y,h,c = LSTM(hidden_size,return_state=True)(y)
    
    y,_,_ = LSTM(hidden_size,return_sequences=True, return_state=True)(x_o,initial_state=[h,c])
    
    y = TimeDistributed(Dense(num_vocab,activation='softmax'))(y)    
    
    model = Model(inputs=[x_i,x_o],outputs=y)
    
    return model

In [89]:
def score_function(expression1,expression2):
    for op in logical_operator.keys():
        expression1 = expression1.replace(str(op),logical_operator[op])
        expression2 = expression2.replace(str(op),logical_operator[op])
    for v1 in [True,False]:
        for v2 in [True,False]:
            sol1=eval(expression1.replace('1',str(v1)).replace('2',str(v2)).strip())
            sol2=eval(expression2.replace('1',str(v1)).replace('2',str(v2)).strip())
            if sol1!=sol2:
                return False
    return True

In [90]:
score_function('1 3 2','2 3 1'),score_function('1 3 2','2 4 1')

(True, False)

In [107]:
encoder_agent = get_encoder_agent()
encoder_agent.summary()

(None, None, 10)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, None, 1)      0                                            
__________________________________________________________________________________________________
time_distributed_20 (TimeDistri (None, None, 1, 10)  40          input_26[0][0]                   
__________________________________________________________________________________________________
lambda_12 (Lambda)              (None, None, 10)     0           time_distributed_20[0][0]        
__________________________________________________________________________________________________
input_27 (InputLayer)           (None, None, 1)      0                                            
____________________________________________________________________________________________

In [108]:
decoder_agent = get_decoder_agent()
decoder_agent.summary()

(None, None, 10)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_28 (InputLayer)           (None, None, 1)      0                                            
__________________________________________________________________________________________________
time_distributed_22 (TimeDistri (None, None, 1, 10)  30          input_28[0][0]                   
__________________________________________________________________________________________________
lambda_13 (Lambda)              (None, None, 10)     0           time_distributed_22[0][0]        
__________________________________________________________________________________________________
input_29 (InputLayer)           (None, None, 1)      0                                            
____________________________________________________________________________________________

In [194]:
decoder_agent.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
encoder_agent.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [196]:
from keras.callbacks import TensorBoard,ReduceLROnPlateau

tb_callback = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [110]:
def generate_expression():
    return [random.choice([1,2]),random.choice([3,4]),random.choice([1,2])]
def simulator_generator_example():
    batch = []
    while True:
        batch.append(generate_expression())
        if len(batch) % batch_size ==0 and len(batch)!=0:
            batch = np.expand_dims(batch,axis=-1)
            yield [batch,np.zeros(batch.shape)]
            batch = []

In [187]:
def make_expression(exp_arr):
    return ' '.join([str(z) for z in exp_arr]).strip()

In [197]:
sim_gen = simulator_generator_example()

for epoch_num in range(num_epoch):
    batch = sim_gen.__next__()
    encoder_output = encoder_agent.predict(batch)
    batch = sim_gen.__next__()
    encoder_output = encoder_agent.predict(batch)
    encoder_output_disctretised = np.argmax(encoder_output,axis=-1)
    encoder_output_disctretised=np.expand_dims(encoder_output_disctretised,axis=-1)
    dummy = np.zeros((encoder_output_disctretised.shape))
    decoder_output=decoder_agent.predict([encoder_output_disctretised,dummy])

    decoder_output=decoder_agent.predict([encoder_output_disctretised,dummy])
    decoder_output_discrete = np.argmax(decoder_output,axis=-1)
    rewards = []
    for idx,(ip,op) in enumerate(zip(batch,decoder_output_discrete)):
        try:
            exp_1=make_expression(ip)
            exp_2=make_expression(op)
            reward = score_function(exp_1,exp_2)
            if reward:
                reward = 1
            else:
                reward = -1
        except:
            reward = -1
        rewards.append(reward)
    encoder_agent.fit(batch,reward,epochs=1,batch_size=lwn(batch),callbacks=[tb_callback])
    encoder_agent.fit([encoder_output_disctretised,dummy],reward,epochs=1,batch_size=lwn(batch),callbacks=[tb_callback])
